In [1]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("res.csv", delimiter=" ", names=["iter", "iter_max", "sat_clause", "all_clause", "filename", "alg"])
df

,iter,iter_max,sat_clause,all_clause,filename,alg
0,100,1500,91,91,uf20-91R/uf20-0110.cnf,prob_sat
1,138,1500,91,91,uf20-91R/uf20-0110.cnf,gsat
2,24,1500,91,91,uf20-91R/uf20-0110.cnf,prob_sat
3,45,1500,91,91,uf20-91R/uf20-0110.cnf,gsat
4,160,1500,91,91,uf20-91R/uf20-0110.cnf,prob_sat
...,...,...,...,...,...,...
4533,425,1500,91,91,uf20-91R/uf20-0250.cnf,gsat
4534,765,1500,91,91,uf20-91R/uf20-0250.cnf,prob_sat
4535,89,1500,91,91,uf20-91R/uf20-0250.cnf,gsat
4536,23,1500,91,91,uf20-91R/uf20-0250.cnf,prob_sat


In [8]:
prob_sat = df[df["alg"] == "prob_sat"]
prob_sat[prob_sat.sat_clause != prob_sat.all_clause]

,iter,iter_max,sat_clause,all_clause,filename,alg
202,1500,1500,84,91,uf20-91R/uf20-0116.cnf,prob_sat
206,1500,1500,86,91,uf20-91R/uf20-0116.cnf,prob_sat
212,1500,1500,87,91,uf20-91R/uf20-0116.cnf,prob_sat
224,1500,1500,83,91,uf20-91R/uf20-0116.cnf,prob_sat
232,1500,1500,79,91,uf20-91R/uf20-0116.cnf,prob_sat
...,...,...,...,...,...,...
4310,1500,1500,84,91,uf20-91R/uf20-0241.cnf,prob_sat
4324,1500,1500,83,91,uf20-91R/uf20-0241.cnf,prob_sat
4406,1500,1500,83,91,uf20-91R/uf20-0250.cnf,prob_sat
4416,1500,1500,87,91,uf20-91R/uf20-0250.cnf,prob_sat
